<a href="https://colab.research.google.com/github/solobala/DLL30/blob/main/DLL30_DZ8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по теме «Рекуррентные сети 3»

Преподаватель: Даниил Корбут, Алексей Миронов, Татьяна Булгакова, Светлана Медведева, Артём Качалкин, Никита Селин

Возьмите англо-русскую пару фраз (www.manythings.org....org/anki/)

1. Обучите на них seq2seq по аналогии с занятием.

2. Оцените полученное качество

3. Попробуйте добавить +1 рекуррентный слой в encoder и decoder

4. Попробуйте заменить GRU ячейки на lstm-ячейки

Оцените качество во всех случаях

*5. Добавить в лекционный ноутбук, в функцию train, обучение батчами.

# Импорт библиотек

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


In [2]:
from google.colab import drive
from google.colab import files

In [3]:
from io import open
import unicodedata
import string
import re
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from itertools import product
plt.switch_backend('agg')
%matplotlib inline

In [4]:
import warnings
warnings.filterwarnings('ignore')

# 1. Загрузка датасета

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip '/content/drive/MyDrive/rus-eng.zip'

Archive:  /content/drive/MyDrive/rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [7]:
!tail rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

#2. Параметры

In [8]:
SEED = 1234
random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device=='cpu':
  torch.manual_seed(SEED)
else:
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

#3. Мета - параметры

In [9]:
SOS_token = 0
EOS_token = 1
lang1 = 'rus'
lang2 = 'eng'
MAX_LENGTH = 10 # Максимальное к-во токенов в предложении

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
rnn_types           = [nn.RNN, nn.LSTM, nn.GRU]
num_hidden_lay_list = [1,2]
cols                = ['RNN_Type', 'Hidden_layers', 'Loss']
PATH               = '/content/drive/My Drive/dz8_'

# 4. Пользовательские функции

## 4.1. Подготовка датасета

In [10]:
class Lang:
    """Грануляция на уровне слов. name - название языка.
    3 вида словарей в классе - Word2index ( слово-индекс), index2word (наоборот) и word2count (слово и кол-во его применений).
    2 метода - добавить слово и добавить предложение.
    При этом, если слово не встречалось ранее, оно добавляется во все словари.
    Если встречалось - значение словаря word2count для данного слова увеличивается на 1"""
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    """Переводит символы Юникода в формат Ascii"""
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [12]:
def normalizeString(s):
    """Переводит в нижний регистр, убирает пробелы слева и справа.
    Добавляет пробел перед .!?, чтобы выделить знаки в отдельные токены.
    Все симводы, помимо букв и .!?, заменяет на пробелы"""
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-ЯёЁ.!?]+", r" ", s)
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [13]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Открывает файл для чтения, испольхуя кодировку utf8,
    # Убирает пробелы слева и справа,
    # делит на строки по '\n'
    lines = open('rus.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Каждую строку разделяет на части с использованием табуляции, в полученном списке использует 2 первых элемента
    pairs = [[normalizeString(s) for s in l.split('\t')][:2] for l in lines] #

    # создает экземпляры класса Lang для обоих языков, при необходимости
    # Можно использовать реверсирование, т.е поменять исходный язык и яык перевода местами
    # подготавливает пары предложений
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [14]:
def filterPair(p, prefixes):
    """Вспомогательная функция: разбивает строку на токены, используя знак пробела,
    Определяет число токенов  в строке для каждого предложения из пары.
    Проверяет, что их число не первышает MAX_LENGTH.
    Проверяет, что предложение на английском начинается с одного из eng_prefixes.
    Возвращает результат всех проверок (True или False)"""
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(prefixes)

In [15]:
def filterPairs(pairs, prefixes):
    """Вспомогательная функция - оставляем только те предложения, где число текенов не более 10"""
    return [pair for pair in pairs if filterPair(pair, prefixes)]

In [16]:
def prepareData(lang1, lang2, prefixes, reverse=False):
    """Функция для поготовки датасета"""
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs, prefixes)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [17]:
def indexesFromSentence(lang, sentence):
    """переводит преложение в последовательность индексов слов"""
    return [lang.word2index[word] for word in sentence.split(' ')]

In [18]:
def tensorFromSentence(lang, sentence):
    """превращает последовательность индексов слов предложения в тензор"""
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # добавляем еще одну размерность
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [19]:
def tensorsFromPair(pair, input_lang, output_lang):
    """Готовит тензоры из последовательностей токенов входного и выходного предложений"""
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## 4.2. Обучение модели

In [20]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, rnn_type):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

In [21]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, rnn_type):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn       = rnn_type(hidden_size, hidden_size, n_layers)
        self.out       = nn.Linear(hidden_size, output_size)
        self.softmax   = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output         = self.embedding(input).view(1, 1, -1)
        output         = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output         = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

In [22]:
def train(input_tensor, target_tensor,
          encoder, decoder, encoder_optimizer, decoder_optimizer,
          criterion, max_length, rnn_type, n_layers, teacher_forcing_ratio):
    """Выполняет итерацию обучения модели
    Args:
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    max_length=MAX_LENGTH - максимальное кол-во токенов  каждом предложении,
    rnn_type - тип RNN ячейки,
    n_layers - число скрытых слоев,
    teacher_forcing_ratio"""
    if rnn_type.__name__ == 'LSTM':
        encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
    else:
        encoder_hidden = encoder.initHidden(n_layers)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [23]:
def asMinutes(s):
    """Вспомогательная функция перевода секунд  в минуты"""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    """ длительность итерации обучения"""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [24]:
def trainIters(pair_sentenses, encoder, decoder, learning_rate, n_iters,
               max_length, rnn_type, num_layers, teacher_forcing_ratio, input_lan, output_lan,
               print_every=5000, plot_every=500):
    """выполняет обучение модели"""
    start               = time.time()
    print_loss_total_loc = 0
    print_loss_avg = 0

    encoder_optimizer = optim.Adam(encoder.parameters())
    decoder_optimizer = optim.Adam(decoder.parameters())

    training_pairs = [tensorsFromPair(random.choice(pair_sentenses),
                                      input_lan, output_lan)
                                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder,
                     encoder_optimizer, decoder_optimizer, criterion,
                     max_length, rnn_type, num_layers, teacher_forcing_ratio)
        print_loss_total_loc += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total_loc / print_every
            print_loss_total_loc = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
    return print_loss_avg


In [25]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## 4.3. Оценка модели

In [26]:
def evaluate(encoder, decoder, sentence, max_length, num_layers,
             rnn_type, inp_lang, out_lang):
    """Оценка качества модели на паре предложений"""
    with torch.no_grad():
        input_tensor = tensorFromSentence(inp_lang, sentence)
        input_length = input_tensor.size()[0]

        if rnn_type.__name__ == 'LSTM':
            encoder_hidden = (encoder.initHidden(num_layers), encoder.initHidden(num_layers))
        else:
            encoder_hidden = encoder.initHidden(num_layers)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [27]:
def evaluateRandomly(pair_sen, encoder, decoder, max_length,
                     _lauers, rnn_, in_lang, o_lang,  n=10):
    """Оценка качества модели на случайно отобранных n парах предложений"""
    for i in range(n):
        pair = random.choice(pair_sen)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], MAX_LENGTH,
                                 _lauers, rnn_, in_lang, o_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [28]:
def test_model(max_length_sent, num_hidden_lay_list, hidden_size = 256,
               learn_rate = 0.006, prefixes=eng_prefixes, reverse=True,
               teach_force_ratio = 0.5,
               num_iters = 50000, every = 1000, num_file = 998):
        """ Функция для обучения и оценки модели"""
        # Инициируем энкодер и декодер
        encoder2 = EncoderRNN(input_lang.n_words, hidden_size, num_layers, rnn_type
                               ).to(device)
        decoder2 = DecoderRNN(hidden_size, output_lang.n_words, num_layers, rnn_type
                               ).to(device)

        print(f'{rnn_type.__name__}, {num_layers} layers\n\ntraining')
        print('===========================')
        # выполняем обучение модели, считаем ф-ю потерь
        loss_ = trainIters(pair_s, encoder2, decoder2, learn_rate, num_iters,
                          max_length_sent, rnn_type, num_layers, teach_force_ratio,
                          input_lang, output_lang, every)

        print('\nevaluation\n')
        # Выполняем оценку обученной модели
        evaluateRandomly(pair_s, encoder2, decoder2, max_length_sent,
                         num_layers, rnn_type, input_lang, output_lang)
        print('----------------------------------------------------------------------')
        string          = [rnn_type.__name__, num_layers, loss_]
        df.loc[len(df)] = string
        file_name       = PATH + str(num_file) + '.csv'
        df.to_csv(file_name)

# 5. Эксперименты

In [29]:
# Инициируем экземпляры классов для обоих языков, готовим датасет
print('Подготовка датасета:')
print('----------------------------------------------------------------------')
input_lang, output_lang, pair_s = prepareData(lang1, lang2, prefixes=eng_prefixes, reverse=True)
# Это пример пары предложений
print(random.choice(pair_s))
# датафрейм для сохранения результатов обучения разных моделей
df = pd.DataFrame(columns=cols)

Подготовка датасета:
----------------------------------------------------------------------
Reading lines...
Read 487600 sentence pairs
Trimmed to 28240 sentence pairs
Counting words...
Counted words:
eng 10119
rus 4289
['на следующеи неделе я иду на концерт .', 'i am going to a concert next week .']


In [ ]:
# Эксперименты для моделей с разными типами RNN ячеек (RNN, GRU, LSTM) и кол-вом скрытых состояний
print('Эксперименты с моделями:')
print('----------------------------------------------------------------------')
num_file = 1
for rnn_type, num_layers in product(rnn_types, num_hidden_lay_list):
  test_model(max_length_sent = MAX_LENGTH, num_hidden_lay_list = num_hidden_lay_list,
           learn_rate = 0.0001, prefixes=eng_prefixes, reverse=True, teach_force_ratio = 0.5,
           num_iters = 20000, every = 1000,
           num_file = num_file)

  num_file += 1

Эксперименты с моделями:
----------------------------------------------------------------------
RNN, 1 layers

training


  0%|          | 0/20000 [00:00<?, ?it/s]

2m 11s (- 41m 45s) (1000 5%) 3.5629
4m 33s (- 41m 4s) (2000 10%) 3.2968
6m 52s (- 38m 59s) (3000 15%) 3.2780
9m 52s (- 39m 30s) (4000 20%) 3.2516
12m 26s (- 37m 18s) (5000 25%) 3.1931
14m 55s (- 34m 50s) (6000 30%) 3.2764
17m 21s (- 32m 14s) (7000 35%) 3.2681
19m 48s (- 29m 42s) (8000 40%) 3.1645
22m 17s (- 27m 14s) (9000 45%) 3.1705
24m 46s (- 24m 46s) (10000 50%) 3.1486
27m 14s (- 22m 17s) (11000 55%) 3.1095
29m 47s (- 19m 51s) (12000 60%) 3.1816
32m 18s (- 17m 23s) (13000 65%) 3.0280
34m 50s (- 14m 55s) (14000 70%) 3.1156
37m 24s (- 12m 28s) (15000 75%) 3.0862
39m 58s (- 9m 59s) (16000 80%) 3.0713
42m 32s (- 7m 30s) (17000 85%) 3.0560
45m 8s (- 5m 0s) (18000 90%) 3.0507
47m 45s (- 2m 30s) (19000 95%) 3.0511
50m 22s (- 0m 0s) (20000 100%) 3.0744

evaluation

> мне надоели гамбургеры .
= i m sick and tired of hamburgers .
< i m not to <EOS>

> поиду покемарю .
= i m going to take a nap .
< i m not to <EOS>

> я быстро учусь .
= i m a fast learner .
< i m not to <EOS>

> я к этому прив

  0%|          | 0/20000 [00:00<?, ?it/s]

2m 19s (- 44m 1s) (1000 5%) 3.6035
4m 39s (- 41m 56s) (2000 10%) 3.3503
7m 2s (- 39m 53s) (3000 15%) 3.2857
9m 28s (- 37m 52s) (4000 20%) 3.3741
11m 56s (- 35m 48s) (5000 25%) 3.2333
14m 24s (- 33m 36s) (6000 30%) 3.2490
16m 54s (- 31m 23s) (7000 35%) 3.2087
19m 26s (- 29m 9s) (8000 40%) 3.2187
22m 0s (- 26m 53s) (9000 45%) 3.1700
24m 35s (- 24m 35s) (10000 50%) 3.2151
27m 12s (- 22m 15s) (11000 55%) 3.2047
29m 49s (- 19m 52s) (12000 60%) 3.1821
32m 24s (- 17m 27s) (13000 65%) 3.0715
35m 2s (- 15m 1s) (14000 70%) 3.1574
37m 41s (- 12m 33s) (15000 75%) 3.1485
40m 22s (- 10m 5s) (16000 80%) 3.0796
43m 4s (- 7m 36s) (17000 85%) 3.1761
45m 48s (- 5m 5s) (18000 90%) 3.1625
48m 29s (- 2m 33s) (19000 95%) 3.0863
51m 11s (- 0m 0s) (20000 100%) 3.0781

evaluation

> я довольно ленив .
= i m quite lazy .
< i m going to . . . <EOS>

> я уже не занят .
= i m not busy anymore .
< i m going to . . . <EOS>

> вы здесь гость .
= you re a guest here .
< you re not talkative . <EOS>

> ты что то задумал

  0%|          | 0/20000 [00:00<?, ?it/s]

2m 56s (- 55m 53s) (1000 5%) 3.2445
5m 56s (- 53m 32s) (2000 10%) 3.0350
8m 59s (- 50m 57s) (3000 15%) 2.8559
12m 5s (- 48m 23s) (4000 20%) 2.8064
15m 14s (- 45m 43s) (5000 25%) 2.7136
18m 27s (- 43m 4s) (6000 30%) 2.6190
21m 41s (- 40m 16s) (7000 35%) 2.5434
24m 54s (- 37m 22s) (8000 40%) 2.5164
28m 9s (- 34m 24s) (9000 45%) 2.5221
31m 24s (- 31m 24s) (10000 50%) 2.4593
34m 40s (- 28m 22s) (11000 55%) 2.3765
37m 56s (- 25m 17s) (12000 60%) 2.3390
41m 14s (- 22m 12s) (13000 65%) 2.3416
44m 34s (- 19m 6s) (14000 70%) 2.3059
47m 52s (- 15m 57s) (15000 75%) 2.3142
51m 12s (- 12m 48s) (16000 80%) 2.2474
54m 33s (- 9m 37s) (17000 85%) 2.1407
57m 54s (- 6m 26s) (18000 90%) 2.2059
61m 17s (- 3m 13s) (19000 95%) 2.1201
64m 39s (- 0m 0s) (20000 100%) 2.1409

evaluation

> ты явно уставшии .
= you re obviously tired .
< you re obviously now . <EOS>

> я поиду сяду .
= i m going to go sit down .
< i m going to be . . <EOS>

> я рад что том в порядке .
= i m glad tom s ok .
< i m glad tom is . . <

  0%|          | 0/20000 [00:00<?, ?it/s]

3m 51s (- 73m 22s) (1000 5%) 3.4575
7m 47s (- 70m 10s) (2000 10%) 3.0444
11m 46s (- 66m 43s) (3000 15%) 2.9215
15m 50s (- 63m 21s) (4000 20%) 2.8428
19m 54s (- 59m 43s) (5000 25%) 2.8025


Из-за сбоя в работе Colab пришлось перезапускать обучение моделией LSTM и GRU

In [30]:
rnn_type = rnn_types[1]
num_layers = 2
test_model(max_length_sent = MAX_LENGTH, num_hidden_lay_list = num_hidden_lay_list,
           learn_rate = 0.0001, prefixes=eng_prefixes, reverse=True, teach_force_ratio = 0.5,
           num_iters = 20000, every = 1000,
           num_file = 4)

LSTM, 2 layers

training


  0%|          | 0/20000 [00:00<?, ?it/s]

0m 44s (- 14m 6s) (1000 5%) 3.4836
1m 20s (- 12m 3s) (2000 10%) 3.0183
1m 52s (- 10m 36s) (3000 15%) 2.9271
2m 22s (- 9m 28s) (4000 20%) 2.8348
2m 51s (- 8m 35s) (5000 25%) 2.7498
3m 21s (- 7m 50s) (6000 30%) 2.7496
3m 52s (- 7m 10s) (7000 35%) 2.7049
4m 22s (- 6m 33s) (8000 40%) 2.5812
4m 52s (- 5m 57s) (9000 45%) 2.5452
5m 22s (- 5m 22s) (10000 50%) 2.5176
5m 51s (- 4m 47s) (11000 55%) 2.4546
6m 21s (- 4m 14s) (12000 60%) 2.4329
6m 51s (- 3m 41s) (13000 65%) 2.3166
7m 20s (- 3m 8s) (14000 70%) 2.3834
7m 50s (- 2m 36s) (15000 75%) 2.3133
8m 20s (- 2m 5s) (16000 80%) 2.2813
8m 49s (- 1m 33s) (17000 85%) 2.2585
9m 19s (- 1m 2s) (18000 90%) 2.2082
9m 48s (- 0m 30s) (19000 95%) 2.2091
10m 18s (- 0m 0s) (20000 100%) 2.1924

evaluation

> мне надоели гамбургеры .
= i m sick and tired of hamburgers .
< i m afraid of the . . <EOS>

> поиду покемарю .
= i m going to take a nap .
< i m going to go . . <EOS>

> я быстро учусь .
= i m a fast learner .
< i m already a . <EOS>

> я к этому привыкла

In [31]:
rnn_type = rnn_types[2]
num_layers = 1
test_model(max_length_sent = MAX_LENGTH, num_hidden_lay_list = num_hidden_lay_list,
           learn_rate = 0.0001, prefixes=eng_prefixes, reverse=True, teach_force_ratio = 0.5,
           num_iters = 20000, every = 1000,
           num_file = 5)

GRU, 1 layers

training


  0%|          | 0/20000 [00:00<?, ?it/s]

0m 24s (- 7m 54s) (1000 5%) 3.3753
0m 48s (- 7m 16s) (2000 10%) 2.9928
1m 12s (- 6m 51s) (3000 15%) 2.8116
1m 36s (- 6m 26s) (4000 20%) 2.7267
2m 0s (- 6m 2s) (5000 25%) 2.5281
2m 25s (- 5m 38s) (6000 30%) 2.5271
2m 48s (- 5m 13s) (7000 35%) 2.4128
3m 12s (- 4m 49s) (8000 40%) 2.3036
3m 37s (- 4m 25s) (9000 45%) 2.2012
4m 1s (- 4m 1s) (10000 50%) 2.1617
4m 25s (- 3m 37s) (11000 55%) 2.2025
4m 50s (- 3m 13s) (12000 60%) 2.0974
5m 13s (- 2m 48s) (13000 65%) 2.0001
5m 38s (- 2m 24s) (14000 70%) 2.0015
6m 2s (- 2m 0s) (15000 75%) 1.9413
6m 27s (- 1m 36s) (16000 80%) 1.8976
6m 51s (- 1m 12s) (17000 85%) 1.8740
7m 16s (- 0m 48s) (18000 90%) 1.8917
7m 40s (- 0m 24s) (19000 95%) 1.8566
8m 5s (- 0m 0s) (20000 100%) 1.8025

evaluation

> я довольно ленив .
= i m quite lazy .
< i m a thirsty . <EOS>

> я уже не занят .
= i m not busy anymore .
< i m not busy . . <EOS>

> вы здесь гость .
= you re a guest here .
< you re the best here . <EOS>

> ты что то задумала не так ли ?
= you re up to someth

In [32]:
rnn_type = rnn_types[2]
num_layers = 2
test_model(max_length_sent = MAX_LENGTH, num_hidden_lay_list = num_hidden_lay_list,
           learn_rate = 0.0001, prefixes=eng_prefixes, reverse=True, teach_force_ratio = 0.5,
           num_iters = 20000, every = 1000,
           num_file = 6)

GRU, 2 layers

training


  0%|          | 0/20000 [00:00<?, ?it/s]

0m 29s (- 9m 22s) (1000 5%) 3.3325
0m 58s (- 8m 43s) (2000 10%) 3.1199
1m 26s (- 8m 12s) (3000 15%) 2.9015
1m 55s (- 7m 42s) (4000 20%) 2.8652
2m 24s (- 7m 14s) (5000 25%) 2.7089
2m 53s (- 6m 45s) (6000 30%) 2.5727
3m 22s (- 6m 15s) (7000 35%) 2.5258
3m 51s (- 5m 46s) (8000 40%) 2.4931
4m 19s (- 5m 17s) (9000 45%) 2.4566
4m 48s (- 4m 48s) (10000 50%) 2.3849
5m 17s (- 4m 20s) (11000 55%) 2.2864
5m 46s (- 3m 51s) (12000 60%) 2.2599
6m 16s (- 3m 22s) (13000 65%) 2.2788
6m 45s (- 2m 53s) (14000 70%) 2.2178
7m 14s (- 2m 24s) (15000 75%) 2.2212
7m 43s (- 1m 55s) (16000 80%) 2.1327
8m 12s (- 1m 26s) (17000 85%) 2.0740
8m 41s (- 0m 57s) (18000 90%) 2.1186
9m 10s (- 0m 28s) (19000 95%) 2.0282
9m 39s (- 0m 0s) (20000 100%) 2.0522

evaluation

> ты явно уставшии .
= you re obviously tired .
< you re obviously strong . <EOS>

> я поиду сяду .
= i m going to go sit down .
< i m going to take some . <EOS>

> я рад что том в порядке .
= i m glad tom s ok .
< i m glad tom is . . <EOS>

> мы все умрем 

# 6. Выводы

In [58]:
results = pd.DataFrame(columns=cols)
for i in range(1,7):
  df = pd.read_csv(f'{PATH}{str(i)}.csv', usecols=cols)
  results = pd.concat([results, df], ignore_index=True)
results.drop_duplicates(inplace=True)
results.sort_values(by='Loss')

,RNN_Type,Hidden_layers,Loss
8,GRU,1,1.802527
11,GRU,2,2.052183
5,LSTM,1,2.140935
6,LSTM,2,2.192365
0,RNN,1,3.074441
2,RNN,2,3.078134


В целом, за указанное количество итераций (20000) ни одна из моделей хорошо не обучилась, величина потерь достаточно значительна.
Если судить по величине Loss, то наилучшие результаты при обучении на выбранном датасете при прочих равных условиях показала модель с ячейкой RNN типа GRU и числом скрытых уровней = 1
